In [32]:
from numpy.linalg import inv,det
from sklearn.metrics import classification_report


In [31]:
import matplotlib as mpl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow.compat.v1 as tf
from sklearn.preprocessing import LabelEncoder
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
import time
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import roc_auc_score
import scipy.stats as st
from sympy.solvers import solve
from sympy import Symbol
from scipy.optimize import fsolve
from sklearn.metrics import fbeta_score,f1_score,balanced_accuracy_score,recall_score,precision_score
from sklearn.neural_network import MLPClassifier
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from cvxopt import matrix
from cvxopt import solvers
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten, Activation, GlobalAveragePooling2D,Conv2D, MaxPooling2D
from sklearn.ensemble import AdaBoostClassifier # AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier # KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier 
from tensorflow.keras.models import Model, Sequential
import math

In [7]:
class Poly_close_solution:
    def __init__(self,kernel='self._poly',penalty = None,gamma=1,degree = 2,coef0=0,alpha=1,beta=1):
        self.W = None
        self.penalty = penalty
        self.d = degree
        self.b = coef0
        self.alpha=alpha
        self.beta=beta
        self.g=gamma
        self.kernel=kernel
        
    def _poly(self,X_1,X_2):
        return (self.g*(X_1.dot(X_2.T))+self.b)**self.d
    def _rbf(self,x,y):
        return np.exp(-self.g*np.sum((x[...,None,:]-y)**2,axis=2))

    def poly_B_matrix(self,X,Y):
        n=Y[Y==1].shape[0]
        X_sample_split=np.array(np.split(X,np.array([X.shape[0]-n])))
        B=np.zeros((X.shape[0],X.shape[0]))
        I=0
        for  m in X_sample_split:
            I+=m.shape[0]
            B[I-m.shape[0]:I,I-m.shape[0]:I]=eval(self.kernel+'(m,m)')/m.shape[0]
        return B
    
    def K_x(self,X,x_t):
        return eval(self.kernel+'(X,x_t)')
    
    def QplusBeta(self,X,Y):
        s=time.time()
        K=eval(self.kernel+'(X,X)')
        H_p=np.diag(np.diagonal(K))
        B_p=self.poly_B_matrix(X,Y)
        Qplus_beta=K+self.alpha*(H_p-B_p)+np.diag([self.beta]*X.shape[0])
        e=time.time()
        return K,Qplus_beta
    def fit(self,invQ,X,x_t):
        
        
        W=invQ.dot(self.K_x(X,x_t))##求逆耗时
        
        return W.reshape(-1)
    def delta_phi(self,K,invQ,X,Y,x_t,j):
        
        w=self.fit(invQ,X,x_t)
        w_noty=w

        w_y=np.zeros((X.shape[0]))
        indice=np.argwhere(Y==j).reshape(-1)
        w_noty[indice]=0
        w_y[indice]=w[indice]
        delta=np.dot(w_y,K.dot(w_y))+np.dot(w_noty,K.dot(w_noty))-2*np.dot(w_y,self.K_x(X,x_t))
        return delta
    def predict(self,K,invQ,X,Y,x_t):
        delta=[]
        for k in np.unique(Y):
            delta.append(self.delta_phi(K,invQ,X,Y,x_t,k))
        return np.argmin(delta)

In [12]:
from sklearn.metrics import fbeta_score
from numpy.linalg import inv,det
metrics_names=['G-mean','Balanced_acc','F1_score','Recall','Precision']
def DRM_FiveCV(X,y,model=Poly_close_solution(
                                )):
    
    G_m,bacc,f2,rec,pre=list(),list(),list(),list(),list()
    for train_index,test_index in cv.split(X,y):
                
                K,Q_beta=model.QplusBeta(X[train_index],y[train_index])
                invQ=inv(Q_beta)
                y_test_label=list()
                for i in range(y[test_index].shape[0]):
                    y_test_label.append(model.predict(K,invQ,X[train_index],y[train_index],X[test_index][i]))

                rec.append(recall_score(y[test_index],y_test_label))
                pre.append(precision_score(y[test_index],y_test_label))
                G_m.append(math.sqrt(recall_score(y[test_index],y_test_label)*recall_score(y[test_index],y_test_label,pos_label=0)))
                bacc.append(balanced_accuracy_score(y[test_index],y_test_label))
                f2.append(fbeta_score(y[test_index],y_test_label,beta=2))
 
    return G_m,bacc,f2,rec,pre


In [22]:
ecoli=pd.read_csv('/Users/lvjingzhe/Desktop/璇/modified_althogram/DRM/data/ecoli_.csv',)
L=[]
for k in ecoli['Class'].values:
    if k in ['0','1','2','3' ]:
        L.append(0)
    else:L.append(1)
ecoli['Class']=L
E=np.array(ecoli)
np.random.seed(4123)
np.random.shuffle(E)
X_sample=E[:,:-1]/np.max(E[:,:-1],axis=0)
y_sample=E[:,-1]
X_sample.shape        

(336, 7)

In [23]:
x_tr, x_te, y_tr, y_te = train_test_split(X_sample,y_sample,test_size = 0.2,
                                                  shuffle = True,
                                                  random_state = 0)
cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=0)


In [25]:
degree=np.array([2,3,4,5,8,10])##tr:te=8:2
gamma=np.array([0.001,0.01,0.1,1,10])
parameter=np.array([0.01,0.1,1,10,100])
X_tr_ord=np.concatenate((x_tr[y_tr==0],x_tr[y_tr==1]))
y_tr_ord=np.concatenate((y_tr[y_tr==0],y_tr[y_tr==1]))
all_results_Ldrm = []
import time
import math
start=time.time()

for k in parameter:
    for p in parameter:
        for d in degree:
            Lo_score=np.mean(DRM_FiveCV(X_tr_ord,y_tr_ord,model=Poly_close_solution(alpha=k,beta=p,degree=d
                                )),axis=1)

            metric_res = {'alpha':k,'beta':p,'degree':d}

            for name, value in zip(metrics_names, Lo_score):
                    metric_res[name] = value


            all_results_Ldrm.append(metric_res)
end=time.time()
print("DRM_L_fivefold-cv time:%.2f"%(end-start))

DRM_L_fivefold-cv time:14.41


In [29]:
eva=pd.DataFrame(all_results_Ldrm)
bias=eva[eva.iloc[:,-4]==(eva.iloc[:,-4]).max()]
alpha,beta,degree=bias[['alpha','beta','degree']].values[0]

alpha,beta,degree

(1.0, 0.01, 10.0)

In [34]:
P_model=Poly_close_solution(alpha=1,beta=0.01,degree=10)
K,Q_beta=P_model.QplusBeta(x_tr,y_tr)
invQ=inv(Q_beta)
predict_y=[]
for i in range(y_te.shape[0]):
    predict_y.append(P_model.predict(K,invQ,x_tr,y_tr,x_te[i]))
print(classification_report(y_te,predict_y))
print('>%s: Average G-mean:%.3f ' % ('DRM(P):',np.sqrt(recall_score(y_te,predict_y)*recall_score(y_te,predict_y,pos_label=0))))
print('>%s: Average Balanced_Acc: %.3f ' % ('DRM(P):',balanced_accuracy_score(y_te,predict_y)))
print('>%s: Average Fbeta: %.3f' % ('DRM(P):',fbeta_score(y_te,predict_y,beta=max(2,np.log(y_tr.shape[0]/sum(y_tr)-1)))))
print('>%s: Average Recall: %.3f' % ('DRM(P):',recall_score(y_te,predict_y)))    


              precision    recall  f1-score   support

           0       0.97      1.00      0.98        63
           1       1.00      0.60      0.75         5

    accuracy                           0.97        68
   macro avg       0.98      0.80      0.87        68
weighted avg       0.97      0.97      0.97        68

>DRM(P):: Average G-mean:0.775 
>DRM(P):: Average Balanced_Acc: 0.800 
>DRM(P):: Average Fbeta: 0.640
>DRM(P):: Average Recall: 0.600
